# LangChain: Q&A over Documents

An example might be a tool that would allow you to query a product catalog for items of interest.

In [ ]:
#pip install --upgrade langchain

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [2]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [3]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.llms import OpenAI

In [4]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

In [5]:
from langchain.indexes import VectorstoreIndexCreator

In [6]:
#pip install docarray

In [7]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [16]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

**Note**:
- The notebook uses `langchain==0.0.179` and `openai==0.27.7`
- For these library versions, `VectorstoreIndexCreator` uses `text-davinci-003` as the base model, which has been deprecated since 1 January 2024.
- The replacement model, `gpt-3.5-turbo-instruct` will be used instead for the `query`.
- The `response` format might be different than the video because of this replacement model.

In [17]:
llm_replacement_model = OpenAI(temperature=0, 
                               model='gpt-3.5-turbo-instruct')

response = index.query(query, 
                       llm = llm_replacement_model)

In [18]:
display(Markdown(response))



| Name | Description | Sun Protection Rating |
| --- | --- | --- |
| Men's Tropical Plaid Short-Sleeve Shirt | Made of 100% polyester, UPF 50+ rating, wrinkle-resistant, front and back cape venting, two front bellows pockets | SPF 50+, blocks 98% of harmful UV rays |
| Men's Plaid Tropic Shirt, Short-Sleeve | Made of 52% polyester and 48% nylon, UPF 50+ rating, SunSmart technology, wrinkle-free, front and back cape venting, two front bellows pockets | SPF 50+, blocks 98% of harmful UV rays |
| Men's TropicVibe Shirt, Short-Sleeve | Made of 71% nylon and 29% polyester, UPF 50+ rating, front and back cape venting, two front bellows pockets | SPF 50+, blocks 98% of harmful UV rays |
| Sun Shield Shirt | Made of 78% nylon and 22% Lycra Xtra Life fiber, UPF 50+ rating, moisture-wicking, abrasion-resistant, fits over swimsuit | SPF 50+, blocks 98% of harmful UV rays |

## Step By Step

In [19]:
from langchain.document_loaders import CSVLoader
loader = CSVLoader(file_path=file)

In [20]:
docs = loader.load()

In [21]:
docs[0]

Document(page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.", metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0})

In [22]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [23]:
embed = embeddings.embed_query("Hi my name is Harrison")

In [24]:
print(len(embed))

1536


In [25]:
print(embed[:5])

[-0.02199048176407814, 0.006746508646756411, -0.018174780532717705, -0.03918623551726341, -0.01404528971761465]


In [26]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [42]:
query = "Please suggest a plaid shirt."

In [43]:
docs = db.similarity_search(query)

In [44]:
len(docs)

4

In [45]:
docs[0]

Document(page_content=": 248\nname: Classic Plaid Short-Sleeve Shirt\ndescription: No matter how warm the days get, this colorful linen shirt will keep you cool and comfortable.\n\nSize & Fit: Slightly Fitted: Relaxed through the chest and sleeve, with a slightly slimmer waist.\n\nWhy We Love It: We love spending summer days outside, where the only air-conditioning is the occasional cool breeze. That's why we use linen made only from pure European flax – one of the lightest, most breathable natural fibers available.\n\nFabric & Care: 100% linen. Machine wash and dry.\n\nAdditional Features: Single patch pocket. Shirttail hem looks great tucked in or worn out. Imported.", metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 248})

In [46]:
retriever = db.as_retriever()

In [47]:
llm = ChatOpenAI(temperature = 0.0, model=llm_model)

In [48]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])


In [49]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
plaid shirts in a table in markdown and summarize each one.") 


In [50]:
display(Markdown(response))

| Name                                     | Description                                                                                                                                                                                                                                                   |
|------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Classic Plaid Short-Sleeve Shirt         | Colorful linen shirt that keeps you cool and comfortable. Slightly fitted for a relaxed fit. Made from 100% linen. Features a single patch pocket and shirttail hem. Machine wash and dry. Imported.                                                               |
| Signature Plaid Oxford Shirt             | Soft, slim-fitting oxford cloth shirt in plaid. Slim fit through the chest, sleeve, and waist. Made from 100% cotton with a 6.5 oz. weight. Garment-washed for softness. Button-down collar, patch pocket, box pleat, and shirttail hem. Machine wash and dry. Imported. |
| Northeast Coast Plaid Shirt, Short-Sleeve | Lightweight shirt in summery plaids made from 100% organic cotton. Slightly fitted for a relaxed fit. Garment washed for softness. Features chambray lining, triple needle stitching, and classic fish-eye buttons. Machine wash and dry. Imported.                     |
| Timeless 1933 Plaid Shirt, Slim Fit      | Modern fit, softer feel, and timeless plaids. Slim fit through the chest, sleeve, and waist. Made from 100% cotton flannel. Brushed and garment-washed for a vintage feel. Features angled flap pockets, spread collar, and shirttail hem. Machine wash and dry. Imported. |

In [51]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [52]:
query =  "Please list all your plaid shirts in a table \
in markdown and summarize each one."

In [53]:
response = qa_stuff.run(query)



> Entering new RetrievalQA chain...

> Finished chain.


In [54]:
display(Markdown(response))

| Name                                | Description                                                                                                                                                                                                                   |
|-------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Classic Plaid Short-Sleeve Shirt    | Colorful linen shirt that keeps you cool and comfortable. Slightly Fitted with a relaxed chest and sleeve. Made from 100% linen. Features a single patch pocket and a shirttail hem. Machine wash and dry. Imported.                    |
| Signature Plaid Oxford Shirt        | Soft, slim-fitting oxford cloth shirt in plaid. Slim Fit through the chest, sleeve, and waist. Made from 100% cotton, 6.5 oz. weight. Garment-washed for softness. Neatly tailored with a precise slim fit. Machine wash and dry. Imported. |
| Northeast Coast Plaid Shirt         | Lightweight, softly textured shirt in summery plaids. Made from 100% organic cotton. Slightly Fitted with a relaxed chest and sleeve. Garment washed for softness. Features chambray lining, triple needle stitching, and fish-eye buttons. Imported. |
| Ocean Breeze Plaid Tee, Short-Sleeve| Short-sleeve shirt in stylish plaids. Relaxed fit made of a stretch blend of 61% nylon, 35% Tencel® Lyocell, and 4% spandex. Moisture-wicking, quick-drying, and abrasion-resistant. Features a chest pocket, long enough sleeves, and a rounded hem. Imported. |

In [55]:
response = index.query(query, llm=llm)

In [56]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])

Reminder: Download your notebook to you local computer to save your work.